# Capstone Project - Moving around the city

### Introduction of the business problem and who would be interested in this project

In case you are traveling for work or just for a few days to a city and you will need to visiting constantly a specific place  on your stay, you will be interested in which hotel you can stay to get on time everytime you need to be in this place. I define this situation in the city of Medellin Colombia, which is a city in a middle of a huge valley of approximate 10 miles of longitude, this analysis can help you to take a decision in which hotel you can stay and minimize the time that will take you to go to this specific place.

### Data and source that will be used to solve the problem.

The project will need information about the geographical coordinates (Latitude and Longitude) of the hotels in Medellin City and a technique to divide in clusters this positions and visualizate in a Map of the city.

I use the Foursquare API to get this coordinates and the technique of K-means to divide the hotels in clusters.

### Methodology section

First we import the libraries that are needed

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


Define your credentials of the Foursquare API

In [3]:
# @hidden_cell
CLIENT_ID = 'LHYSLQTS5JFB13FRSBU1DNNGB3C4DQ1L43Z05TF1LLCTE04B' # your Foursquare ID
CLIENT_SECRET = '4NX3B2H2UDQCFSFZYTGD3NKSV0CUFWPJCUEPGC2OHOIJ0VNS' # your Foursquare Secret
VERSION = '20180604'

Get the location of Medellin city and show a Map

In [4]:
address = 'Medellin, Colombia'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Medellin are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Medellin are 6.2443382, -75.573553.


In [5]:
map_medellin = folium.Map(location=[latitude, longitude], zoom_start=12)

map_medellin

Then look for the information of the hotels in the city with the Foursquare API

In [6]:
search_query = 'Hotel'
LIMIT = 300 #Venues
radius = 23000 #Meters
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LHYSLQTS5JFB13FRSBU1DNNGB3C4DQ1L43Z05TF1LLCTE04B&client_secret=4NX3B2H2UDQCFSFZYTGD3NKSV0CUFWPJCUEPGC2OHOIJ0VNS&ll=6.2443382,-75.573553&v=20180604&query=Hotel&radius=23000&limit=300'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3d93e5539a1348c505e252'},
 'response': {'venues': [{'id': '4d0e5651903d37045066c955',
    'name': 'Hotel Nutibara',
    'location': {'address': 'Calle 52A # 50 - 46',
     'lat': 6.252247701061798,
     'lng': -75.56742446342321,
     'labeledLatLngs': [{'label': 'display',
       'lat': 6.252247701061798,
       'lng': -75.56742446342321}],
     'distance': 1111,
     'cc': 'CO',
     'city': 'Medellín',
     'state': 'Antioquia',
     'country': 'Colombia',
     'formattedAddress': ['Calle 52A # 50 - 46',
      'Medellín, Antioquia',
      'Colombia']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597871225',
    'hasPerk': False},
   {'id': '4f3a9360e4b06ff103399c4e',
    'name': 'Hotel 33 Conquest',
    

Extract in a table just the information that its needed

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
print(dataframe.shape)
dataframe.head()

(50, 19)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.postalCode,location.neighborhood
0,4d0e5651903d37045066c955,Hotel Nutibara,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597871225,False,Calle 52A # 50 - 46,6.252248,-75.567424,"[{'label': 'display', 'lat': 6.252247701061798...",1111,CO,Medellín,Antioquia,Colombia,"[Calle 52A # 50 - 46, Medellín, Antioquia, Col...",NaN,NaN,NaN,NaN
1,4f3a9360e4b06ff103399c4e,Hotel 33 Conquest,"[{'id': '4bf58dd8d48988d1f8931735', 'name': 'B...",v-1597871225,False,Carrera 65 numero 33 34,6.239903,-75.582979,"[{'label': 'display', 'lat': 6.23990334227505,...",1153,CO,Medellín,Antioquia,Colombia,[Carrera 65 numero 33 34 (Por el puente de la ...,Por el puente de la 65,NaN,NaN,NaN
2,4d8b44046ce6a35d74617142,Hotel GHL Comfort San Diego,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597871225,False,Clle 31 No 43 90,6.231251,-75.569698,"[{'label': 'display', 'lat': 6.231250567215761...",1518,CO,Medellín,Antioquia,Colombia,"[Clle 31 No 43 90, Medellín, Antioquia, Colombia]",NaN,88090177,NaN,NaN
3,4e812793be7b90cb295ad91e,Hotel El Porton De San Joaquin,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597871225,False,NaN,6.247125,-75.587399,"[{'label': 'display', 'lat': 6.247124686211412...",1563,CO,Medellín,Antióquia,Colombia,"[Medellín, Antióquia, Colombia]",NaN,NaN,NaN,NaN
4,4bc8b365c1c6eee1f8d8fd92,Hotel Mediterráneo Medellín,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597871225,False,Carrera 70 No C5-23,6.247575,-75.589297,"[{'label': 'display', 'lat': 6.247574713234023...",1779,CO,Medellín,Antioquia,Colombia,"[Carrera 70 No C5-23, Medellín, Antioquia, Col...",NaN,NaN,NaN,NaN


In [9]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Hotel Nutibara,Hotel,Calle 52A # 50 - 46,6.252248,-75.567424,"[{'label': 'display', 'lat': 6.252247701061798...",1111,CO,Medellín,Antioquia,Colombia,"[Calle 52A # 50 - 46, Medellín, Antioquia, Col...",NaN,NaN,NaN,4d0e5651903d37045066c955
1,Hotel 33 Conquest,Bed & Breakfast,Carrera 65 numero 33 34,6.239903,-75.582979,"[{'label': 'display', 'lat': 6.23990334227505,...",1153,CO,Medellín,Antioquia,Colombia,[Carrera 65 numero 33 34 (Por el puente de la ...,Por el puente de la 65,NaN,NaN,4f3a9360e4b06ff103399c4e
2,Hotel GHL Comfort San Diego,Hotel,Clle 31 No 43 90,6.231251,-75.569698,"[{'label': 'display', 'lat': 6.231250567215761...",1518,CO,Medellín,Antioquia,Colombia,"[Clle 31 No 43 90, Medellín, Antioquia, Colombia]",NaN,NaN,NaN,4d8b44046ce6a35d74617142
3,Hotel El Porton De San Joaquin,Hotel,NaN,6.247125,-75.587399,"[{'label': 'display', 'lat': 6.247124686211412...",1563,CO,Medellín,Antióquia,Colombia,"[Medellín, Antióquia, Colombia]",NaN,NaN,NaN,4e812793be7b90cb295ad91e
4,Hotel Mediterráneo Medellín,Hotel,Carrera 70 No C5-23,6.247575,-75.589297,"[{'label': 'display', 'lat': 6.247574713234023...",1779,CO,Medellín,Antioquia,Colombia,"[Carrera 70 No C5-23, Medellín, Antioquia, Col...",NaN,NaN,NaN,4bc8b365c1c6eee1f8d8fd92
5,Gran Hotel ****,Hotel,NaN,6.251117,-75.562975,"[{'label': 'display', 'lat': 6.251117353477348...",1392,CO,Medellín,Antioquia,Colombia,"[Medellín, Antioquia, Colombia]",NaN,NaN,NaN,4e0f435662e1d5a30d094c8b
6,Hotel el Rami,Hotel,Carrera 72 #circular 5 - 11,6.246856,-75.591606,"[{'label': 'display', 'lat': 6.2468557, 'lng':...",2017,CO,Medellín,Antioquia,Colombia,"[Carrera 72 #circular 5 - 11, Medellín, Antioq...",NaN,050031,Bolivariana,5a998bffa92d9842d47a95d4
7,Hotel Sauces del Estadio,Hotel,Cra 69 # 49A - 30,6.257425,-75.585480,"[{'label': 'display', 'lat': 6.257425252836906...",1965,CO,Medellín,Antioquia,Colombia,"[Cra 69 # 49A - 30, Medellín, Antioquia, Colom...",NaN,NaN,NaN,4d81916d59d16dcb5d00fc1b
8,Hotel Plaza Mayor,Bed & Breakfast,San Juan 51 # 1-99,6.244671,-75.571328,"[{'label': 'display', 'lat': 6.244670593598062...",249,CO,Medellín,Antióquia,Colombia,"[San Juan 51 # 1-99, Medellín, Antióquia, Colo...",NaN,NaN,NaN,4e59b11bb993732579edb4e4
9,Hotel Tryp Medellin,Hotel,Calle 50 50 702,6.258960,-75.586973,"[{'label': 'display', 'lat': 6.2589601, 'lng':...",2203,CO,Medellín,Antioquia,Colombia,"[Calle 50 50 702, Medellín, Antioquia, Colombia]",NaN,050034,NaN,4eb53656469073bbc608de7b


In [10]:
med_hotels = dataframe_filtered.drop(['address','distance','cc','labeledLatLngs','city','state',
                                      'country','formattedAddress','crossStreet', 'postalCode',
                                      'neighborhood','id'],1)
print(med_hotels.shape)
med_hotels.head(10)

(50, 4)


,name,categories,lat,lng
0,Hotel Nutibara,Hotel,6.252248,-75.567424
1,Hotel 33 Conquest,Bed & Breakfast,6.239903,-75.582979
2,Hotel GHL Comfort San Diego,Hotel,6.231251,-75.569698
3,Hotel El Porton De San Joaquin,Hotel,6.247125,-75.587399
4,Hotel Mediterráneo Medellín,Hotel,6.247575,-75.589297
5,Gran Hotel ****,Hotel,6.251117,-75.562975
6,Hotel el Rami,Hotel,6.246856,-75.591606
7,Hotel Sauces del Estadio,Hotel,6.257425,-75.585480
8,Hotel Plaza Mayor,Bed & Breakfast,6.244671,-75.571328
9,Hotel Tryp Medellin,Hotel,6.258960,-75.586973


Show the locations into a map

In [11]:
map_medellin = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker([lat, lng], radius=2, color='blue', 
                                 popup=label, fill=True, fill_color='red', 
                                 fill_opacity=0.5).add_to(map_medellin)
    

map_medellin

Then import the libraries we need to divide the locations into clusters and add to our table the Cluster label.

In [12]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [13]:
k = 5
Hotels_clustering = med_hotels.drop(['name','categories'],1)
kmeans = KMeans(n_clusters = k, random_state=0).fit(Hotels_clustering)
print(dir(kmeans))
print(len(kmeans.labels_))
print(kmeans.labels_)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_check_test_data', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_n_threads', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_transform', '_validate_data', 'algorithm', 'cluster_centers_', 'copy_x', 'fit', 'fit_predict', 'fit_transform', 'get_params', 'inertia_', 'init', 'labels_', 'max_iter', 'n_clusters', 'n_features_in_', 'n_init', 'n_iter_', 'n_jobs', 'precompute_distances', 'predict', 'random_state', 'score', 'set_params', 'tol', 'transform', 'verbose']
50
[1 2 4 2 2 1 2 3 1 3 3 1 2 0 4 1 1 4 1 1 1 1 2 1 1 1 2 2 2 0 0 0 1 1 1 0 2
 1 1 2 2 1 2 0 1 1 0 1 1 1]


In [22]:
med_hotels['Cluster_labels'] = kmeans.labels_
med_hotels.sort_values(by= 'Cluster_labels', ascending=True, axis=0, inplace=True)
med_hotels.head(10)

,name,categories,lat,lng,Cluster_labels
9,Hotel Tryp Medellin,Hotel,6.258960,-75.586973,0
20,Hotel Linton,Hotel,6.245569,-75.565846,0
26,Hotel Spiwak,Hotel,6.243015,-75.582685,0
8,Hotel Plaza Mayor,Bed & Breakfast,6.244671,-75.571328,0
1,Hotel 33 Conquest,Bed & Breakfast,6.239903,-75.582979,0
37,Hotel Gallery,Hotel,6.243773,-75.563290,0
6,Hotel el Rami,Hotel,6.246856,-75.591606,0
46,Hotel Poblado Boutique Express,Hotel,6.207089,-75.565102,1
2,Hotel GHL Comfort San Diego,Hotel,6.231251,-75.569698,1
5,Gran Hotel ****,Hotel,6.251117,-75.562975,1


Now that are defined our clusters, we can show they into the Map of the city

In [15]:
map_medellin = folium.Map(location=[latitude, longitude], zoom_start=13)


# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(med_hotels['lat'], med_hotels['lng'], med_hotels['name'], med_hotels['Cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_medellin)

map_medellin

Another way to see the clusters in the map

In [16]:
map_medellin = folium.Map(location=[latitude, longitude], zoom_start=13, tiles='Stamen Toner')


# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(med_hotels['lat'], med_hotels['lng'], med_hotels['name'], med_hotels['Cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_medellin)


map_medellin

### Results section

Now we examine each cluster and determine by the positon of your work or visiting place which hotels can help to minimize your time of transportation.

Cluster 1 (Red)

In [17]:
med_hotels.loc[med_hotels['Cluster_labels']==0, med_hotels.columns[[0] + list(range(1, med_hotels.shape[1]))]]

,name,categories,lat,lng,Cluster_labels
46,Hotel Poblado Boutique Express,Hotel,6.207089,-75.565102,0
29,Hotel Loyds,Hotel,6.208798,-75.572760,0
43,Hotel Portón Medellín,Hotel,6.195577,-75.575512,0
30,Hotel Estelar Blue,Hotel,6.202240,-75.570989,0
31,Hotel Du Parc,Hotel,6.209457,-75.569427,0
13,Hotel Park 10 Medellin,Hotel,6.211936,-75.565126,0
35,Hotel Site,Hotel,6.207023,-75.575165,0


Cluster 2 (Purple)

In [18]:
med_hotels.loc[med_hotels['Cluster_labels']==1, med_hotels.columns[[0] + list(range(1, med_hotels.shape[1]))]]

,name,categories,lat,lng,Cluster_labels
0,Hotel Nutibara,Hotel,6.252248,-75.567424,1
48,Hotel El Cid,Hotel,6.253493,-75.565544,1
25,Hotel Suite 45,Hotel,6.247427,-75.564903,1
32,Hotel Boyaca,Hostel,6.253976,-75.572095,1
34,Hotel Internacional,Building,6.249727,-75.565319,1
23,Hotel Medellín Kapital,Hotel,6.252699,-75.575360,1
37,Hotel Gallery,Hotel,6.243773,-75.563290,1
38,Hotel Eupacla,Hotel,6.252389,-75.566443,1
41,Hotel Normandia,Hotel,6.252097,-75.566075,1
44,Hotel Botero Plaza,Hotel,6.253021,-75.567160,1


Cluster 3 (Blue)

In [19]:
med_hotels.loc[med_hotels['Cluster_labels']==2, med_hotels.columns[[0] + list(range(1, med_hotels.shape[1]))]]

,name,categories,lat,lng,Cluster_labels
1,Hotel 33 Conquest,Bed & Breakfast,6.239903,-75.582979,2
3,Hotel El Porton De San Joaquin,Hotel,6.247125,-75.587399,2
4,Hotel Mediterráneo Medellín,Hotel,6.247575,-75.589297,2
42,Hotel Agualongo - Pasto,Hotel,6.242217,-75.584806,2
6,Hotel el Rami,Hotel,6.246856,-75.591606,2
40,Hotel Capilla del Mar,Hotel,6.241770,-75.584420,2
39,Hotel Guadalajara BugA,Hotel,6.241770,-75.584420,2
22,Hotel Platinum Plaza,Hotel Bar,6.239616,-75.580797,2
36,Hotel madison,Residential Building (Apartment / Condo),6.244201,-75.583692,2
12,Hotel Casa Laurel,Bed & Breakfast,6.242704,-75.599238,2


Cluster 4 (Green)

In [20]:
med_hotels.loc[med_hotels['Cluster_labels']==3, med_hotels.columns[[0] + list(range(1, med_hotels.shape[1]))]]

,name,categories,lat,lng,Cluster_labels
10,Hotel Manager Obelisk,Hotel,6.258616,-75.591831,3
7,Hotel Sauces del Estadio,Hotel,6.257425,-75.585480,3
9,Hotel Tryp Medellin,Hotel,6.258960,-75.586973,3


Cluster 5 (Orange)

In [21]:
med_hotels.loc[med_hotels['Cluster_labels']==4, med_hotels.columns[[0] + list(range(1, med_hotels.shape[1]))]]

,name,categories,lat,lng,Cluster_labels
14,Hotel Dann Carlton Belfort,Hotel,6.218177,-75.568700,4
2,Hotel GHL Comfort San Diego,Hotel,6.231251,-75.569698,4
17,LQ Hotel by La Quinta,Hotel,6.224661,-75.565661,4


### Discusion section

This project is based on the Foursquare API, so we depend on the update and use of this application in the city, so a compare the project information with a simple search of hotels with google maps in Medellin city, it is a very similar information but it is more updated Google maps.

### Conclusion section

Based on the information we have although it is not the most updated with all of the new hotels, this project still can help someone to look for a place to stay with the clustering results and minimize its time to get around a place in the city.